Q1.5 
This query aims to expand the FA-ensemblage gene set based on upstream TF binding site motif patterns.

Data  
 - FA gene sets  'https://github.com/NCATS-Tangerine/cq-notebooks/tree/master/FA_gene_sets'
 - Jaspar motif simalarity sparql endpoint at https://tfbsmotif.ncats.io


In [1]:
import csv
import yaml
import requests
import json
import re
import os

NCATS blazegraph instance is up.

Untill the hackathon blazegraph instance was available    
I ran a local instance [described here](https://github.com/NCATS-Tangerine/cq-notebooks/blob/master/Orange_Demonstrator_1_CQs/OrangeQ1.5_Regulatory_Motif_Signature/LocalBlazeGraph.ipynb)


In [2]:
#bg_host = 'http://localhost:9999'
bg_host = 'https://tfbsmotif.ncats.io'
bg = bg_host + '/blazegraph/sparql?format=json&query=' 

which offers:  

    rdf:Description rdf:nodeID="service"
        rdf:type 
            rdf:resource="http://www.w3.org/ns/sparql-service-description#Service"
        endpoint 
            rdf:resource="http://localhost:9999/blazegraph/sparql"
        endpoint 
            rdf:resource="http://localhost:9999/bigdata/LBS/sparql"
            
Trying the blazegraph/sparql endpoint yields

In [3]:
x = requests.get(bg + 'SELECT ?g1 WHERE{?g1 <http://purl.obolibrary.org/obo/SO_adjacent_to> ?o} LIMIT 1')
print(x.text)

{
  "head" : {
    "vars" : [ "g1" ]
  },
  "results" : {
    "bindings" : [ {
      "g1" : {
        "type" : "uri",
        "value" : "http://www.ncbi.nlm.nih.gov/gene/6309"
      }
    } ]
  }
}


Blazegraph is working localy and remotly,   
there is a process in place to refresh it 
when I update the Jaspar RDF.
We are using the RDF stored in Blazegraph to address the 
Fanconi Anemia transcription factor binding site 
motif similarity question in this notebook.

In [4]:
# We want to use existing Monarch Dipper translation tables when constructing SPARQL queries
# 
yamlurl='https://raw.githubusercontent.com/TomConlin/Jaspar_FA/master/translation_tables/curie_map.yaml'
rsponse = requests.get(yamlurl)
PREFIX = yaml.load(rsponse.text)
# print(PREFIX)

yamlurl='https://raw.githubusercontent.com/TomConlin/Jaspar_FA/master/translation_tables/translation_table.yaml'
rsponse = requests.get(yamlurl)
TT = yaml.load(rsponse.text)
# print(TT)

# redecorate the curie base IRI map 
# to look like a sparql prefix namespace map  
# except the bnode: skolemIRI which is given a java 
#     "org.openrdf.query.MalformedQueryException:
# for no good reason

prefixns = ""
for p in PREFIX: 
    if len(p) > 1:
       prefixns += 'PREFIX ' + p + ': <' + PREFIX[p] + '>\n'

print(prefixns)

PREFIX GENO: <http://purl.obolibrary.org/obo/GENO_>
PREFIX RO: <http://purl.obolibrary.org/obo/RO_>
PREFIX SO: <http://purl.obolibrary.org/obo/SO_>
PREFIX SWO: <http://www.ebi.ac.uk/efo/swo/SWO_>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX OIO: <http://www.geneontology.org/formats/oboInOwl#>
PREFIX NCBIGene: <http://www.ncbi.nlm.nih.gov/gene/>
PREFIX JASPAR: <http://fantom.gsc.riken.jp/5/sstar/JASPAR_motif:>



The Fanconi Anemia genes come as symbols/aliases in three sets [here](https://docs.google.com/spreadsheets/d/1yX-5sfrC3vrahf4_k7-5rl4Oqzm853ollIMmUo1PTc0)  
I converted them to NCBI gene identifiers and current symbols [here](https://github.com/NCATS-Tangerine/cq-notebooks/tree/master/FA_gene_sets)

If you need the FA gene sets from a different repo you can use   
 - "https://raw.githubusercontent.com/NCATS-Tangerine/cq-notebooks/master/FA_gene_sets/FA_1_core_complex.txt"
 - "https://raw.githubusercontent.com/NCATS-Tangerine/cq-notebooks/master/FA_gene_sets/FA_2_effector_proteins.txt"
 - "https://raw.githubusercontent.com/NCATS-Tangerine/cq-notebooks/master/FA_gene_sets/FA_3_associated_proteins.txt" 
 
but here they are just in a parent directory  
(unless someone decides to reorganize the directories)

build a data structure to hold 
 - the (three) sets of fa genes 
 - the collection of hits for each fa gene   
maybe something like:  
    
```
fs_sets {
    set_name: {
        fa_curie: {
            fa_symbol:,
            hits: {
                curie: {
                    iri:,
                    symbol:,
                    score:
                },
                ...
            }  
        }, ...
  }, ...
}
```

In [5]:
# because I put numbers in the filenames to order them
fa_sets={
    'core_complex' : {'num': '1'},
    'effector_proteins' :  {'num': '2'},
    'associated_proteins' : {'num': '3'}
}

# keep a handy list too
fagenes = []

for fa_set in fa_sets:
    path = '../../FA_gene_sets/FA_' + fa_sets[fa_set]['num'] + '_' + fa_set + '.txt'
    with open(path, 'r') as tabfile:
        filereader = csv.reader(tabfile, delimiter='\t')
        fa_sets[fa_set].pop('num') # was just there for file path
        for row in filereader:   
            (fa_curie, fa_symbol) = row
            fagenes.append(fa_curie)
            fa_sets[fa_set][fa_curie]= {
                    'symbol': fa_symbol, 
                    'iri' :   re.sub('NCBIGene:',PREFIX['NCBIGene'],fa_curie),
                    'hits' : {}
            }
    print('In the ' + fa_set + ' set we found  \t' + str(len(fa_sets[fa_set])) + ' genes')


In the effector_proteins set we found  	11 genes
In the associated_proteins set we found  	5 genes
In the core_complex set we found  	11 genes


In [6]:
# sanity check input
for fas in fa_sets:
    print(fas + " includes:")
    for fa in fa_sets[fas]:
        print("\t" + fa_sets[fas][fa]['symbol'] )

core_complex includes:
	FANCA
	FANCB
	FANCC
	FANCE
	FANCF
	FANCG
	FANCL
	FANCM
	FANCD2
	FANCI
	UBE2T
effector_proteins includes:
	BRCA2
	BRIP1
	PALB2
	RAD51C
	SLX4
	ERCC4
	RAD51
	BRCA1
	MAD2L2
	XRCC2
	RFWD3
associated_proteins includes:
	FAAP100
	FAAP24
	FAAP20
	CENPS
	CENPX


We can get all the triples patterns which are available in 'jaspar.nt' from  
this [GraphViz dot file](https://github.com/TomConlin/Jaspar_FA/blob/master/jaspar_target_model.gv) since we used it to guide generating the RDF data in the first place.  

![](https://github.com/TomConlin/Jaspar_FA/blob/master/jaspar_target_model.png?raw=true)



Composing the query by atomizing the GraphViz elements   
and selectivly translating with the same tables   
the data was generates with means  
the SPARQL query remains free of semanticly obsolete clauses.

I did this manually, but in general, the rules were roughly:
  
-    remove angle brackets
-    add trailing dot 
-    replace BNODE: with __?__
-    if the element (predicate) is a curie then wrap it in a TT lookup
-    if the object is a LITERAL wrap it in quotes
-    give the items of intrest specific __?names__
-    formating to suit sensibilities

In particular we want: 
    candidate genes which share motif similarity with FA genes
    
    <NCBIGene:fagene><SO:adjacent_to><BNODE:gene1_upstream_region>
    <BNODE:gene1_upstream_region><RO:member of><BNODE:pairwise_similarity>
    <BNODE:gene2_upstream_region><RO:member of><BNODE:pairwise_similarity>
    <NCBIGene:xyz><SO:adjacent_to><BNODE:gene2_upstream_region> 
    
    when <NCBIGene:xyz> is not <NCBIGene:fagene> 
    
we may also be interested in limiting by region extent size   
or Jaccard similarity score

    <BNODE:pairwise_similarity><SWO:Similarity score><0.73>
    <BNODE:gene1_upstream_region><GENO:has_extent><1k>
    
    

Much of the effort to this point has been developing and maintaining a context  
in which we are able to __write a readable query__.  

Here, given a (fa) gene, we are looking for other genes   
with optimal matches (similarity=1) within their 1k start regions.

In [6]:
selectstr = ' '.join([
    'SELECT ?fagene ?candidate\n',
    'WHERE{\n',
        '?fagene', 'SO:adjacent_to', '?gene1_upstream_region .\n', 
        '?gene1_upstream_region',  TT['RO:member of'], '?pairwise_similarity .\n',
        '?gene2_upstream_region',  TT['RO:member of'], '?pairwise_similarity .\n',
        'FILTER(?gene1_upstream_region != ?gene2_upstream_region) \n', 
        '?candidate', 'SO:adjacent_to', '?gene2_upstream_region .\n',
        '?pairwise_similarity', TT['SWO:Similarity score'], "'1' .\n", 
        '?gene1_upstream_region', TT['GENO:has_extent'], "'1k' .\n",
    'FILTER(?fagene != ?candidate)\n}'
    ]) 

# note the absense of opaque identifiers the query engine actually uses

query_1 = prefixns + "\n" + selectstr
print(query_1)

PREFIX GENO: <http://purl.obolibrary.org/obo/GENO_>
PREFIX RO: <http://purl.obolibrary.org/obo/RO_>
PREFIX SO: <http://purl.obolibrary.org/obo/SO_>
PREFIX SWO: <http://www.ebi.ac.uk/efo/swo/SWO_>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX OIO: <http://www.geneontology.org/formats/oboInOwl#>
PREFIX NCBIGene: <http://www.ncbi.nlm.nih.gov/gene/>
PREFIX JASPAR: <http://fantom.gsc.riken.jp/5/sstar/JASPAR_motif:>

SELECT ?fagene ?candidate
 WHERE{
 ?fagene SO:adjacent_to ?gene1_upstream_region .
 ?gene1_upstream_region RO:0002350 ?pairwise_similarity .
 ?gene2_upstream_region RO:0002350 ?pairwise_similarity .
 FILTER(?gene1_upstream_region != ?gene2_upstream_region) 
 ?candidate SO:adjacent_to ?gene2_upstream_region .
 ?pairwise_similarity SWO:0000425 '1' .
 ?gene1_upstream_region GENO:0000678 '1k' .
 FILTER(?fagene != ?candidate)
}


In [7]:
# apply our query to each gene in just the FA Core complex
candidate_set = {}
core_complex = fa_sets['core_complex']
for fagene in core_complex: 
    payload = {
        'format' : 'json', 
        # '$fagene':  fagene, 
        # BlazeGraph is not accepting the curie. Wants IRI or LITERAL
        '$fagene': '<' + re.sub('NCBIGene:',PREFIX['NCBIGene'],fagene) + '>',
        'query': query_1
    }
    response = requests.post(bg_host + '/blazegraph/sparql', data=payload)
 
    resp = json.loads(response.text)
    if resp['results']['bindings'] != []:
        print(fagene, '\n', 
              re.sub('NCBIGene:', PREFIX['NCBIGene'],fagene))
        candidate_set[fagene] = []
        for hit in resp['results']['bindings']:
            candidate_set[fagene].append(hit['candidate']['value'])
            print('\t', hit['candidate']['value']) 

NCBIGene:29089 
 http://www.ncbi.nlm.nih.gov/gene/29089
	 http://www.ncbi.nlm.nih.gov/gene/3608
	 http://www.ncbi.nlm.nih.gov/gene/171392
	 http://www.ncbi.nlm.nih.gov/gene/340252
	 http://www.ncbi.nlm.nih.gov/gene/8125
	 http://www.ncbi.nlm.nih.gov/gene/54958


UBE2T (NCBIGene:29089) is the only gene in the Fancomi Anemia core complex  
with __optimal__ motif similarity with other human genes  
Briefly those associated genes are:   

- ILF2 http://www.ncbi.nlm.nih.gov/gene/3608  
    a transcription factor required for T-cell expression of the interleukin 2 gene

- ZNF675 https://www.ncbi.nlm.nih.gov/gene/171392  
    the novel zinc finger protein TIZ may play a role during osteoclast differentiation by modulating TRAF6 signaling activity.

- ZNF680 http://www.ncbi.nlm.nih.gov/gene/340252  
    obsevered expressed in thyroid but fairly uncharaterizied

- ANP32A http://www.ncbi.nlm.nih.gov/gene/8125  
    expressed lymph nodes & bone marrow 
    The tumor suppressor acidic nuclear phosphoprotein 32 family, member A 

- TMEM160 http://www.ncbi.nlm.nih.gov/gene/54958  
    Not much to see here

----
Where to go next

- We can look for ideal similarity in larger upstream regions but counter intiutivly larger regions average fewer associations because the number of distinct motifs between the larger regions grows faster than the number of motifs the regions will have in common.  (and I dropped any that dipped below the negotiable threshold of, one part in five)
- We can look for less ideal similarity in the same 1k region 
- We can look for less ideal similarity in larger regions  

For now I am going with looking for some similarity within the same 1k start regions.

In [8]:
# small changes to expose the similarity score
selectstr = ' '.join([
    'SELECT ?fagene ?candidate ?score\n',
    'WHERE{\n',
        '?fagene', 'SO:adjacent_to', '?gene1_upstream_region .\n', 
        '?gene1_upstream_region',  TT['RO:member of'], '?pairwise_similarity .\n',
        '?gene2_upstream_region',  TT['RO:member of'], '?pairwise_similarity .\n',
        'FILTER(?gene1_upstream_region != ?gene2_upstream_region) \n', 
        '?candidate', 'SO:adjacent_to', '?gene2_upstream_region .\n',
        '?pairwise_similarity', TT['SWO:Similarity score'], '?score .\n', 
        '?gene1_upstream_region', TT['GENO:has_extent'], "'1k' .\n",
    'FILTER(?fagene != ?candidate)\n',
    #'FILTER(xsd:float(?score) >= 0.5)\n',
    '}\n',
    'ORDER by DESC(?score)\n'
    ])
query = prefixns + "\n" + selectstr
print(selectstr)

SELECT ?fagene ?candidate ?score
 WHERE{
 ?fagene SO:adjacent_to ?gene1_upstream_region .
 ?gene1_upstream_region RO:0002350 ?pairwise_similarity .
 ?gene2_upstream_region RO:0002350 ?pairwise_similarity .
 FILTER(?gene1_upstream_region != ?gene2_upstream_region) 
 ?candidate SO:adjacent_to ?gene2_upstream_region .
 ?pairwise_similarity SWO:0000425 ?score .
 ?gene1_upstream_region GENO:0000678 '1k' .
 FILTER(?fagene != ?candidate)
 }
 ORDER by DESC(?score)



In [9]:
# query the sparql endpoint with each of the FA genes in all the sets
# for other genes with similar motif patterns in their upstream regions 
for fa_set in fa_sets:
    for fagene in fa_sets[fa_set]:
        payload = {
            'format' : 'json', 
            '$fagene': '<' + re.sub('NCBIGene:',PREFIX['NCBIGene'],fagene) + '>',
            'query'  : query
        }
        response = requests.post(bg_host + '/blazegraph/sparql', data=payload)
        # print(response)
        resp = json.loads(response.text)
        # print(response.text)
    
        if resp['results']['bindings'] != []:
            # print(fa_sets[fa_set][fagene], '\t', 
              # re.sub('NCBIGene:', PREFIX['NCBIGene'], fagene))
            fa_sets[fa_set][fagene]['hits'] = {}
        
            for hit in resp['results']['bindings']:
                fa_sets[fa_set][fagene]['hits'][ hit['candidate']['value'] ] = {
                    'score': hit['score']['value']}
                # print('\t', hit['candidate']['value'], hit['score']['value'])

In [10]:
# Researchers would rather to see gene symbols. 
# Here we are relying on HGNC 
# for up to date human gene nomenclature.

def entrez_symbol(ncbigene_uri):
    api_uri = re.sub(
        'http://www.ncbi.nlm.nih.gov/gene/',
        'http://rest.genenames.org/search/entrez_id/', ncbigene_uri)
    response = requests.get(api_uri, headers={'Accept': 'application/json'})
    if str(response) == '<Response [200]>':
        hgnc = json.loads(response.text)
        if hgnc['response']['numFound'] > 0:
            symbol = hgnc['response']['docs'][0]['symbol']
        else:
            symbol = ""  # none found    
    else: # it will be ugly so someone will notice
        symbol = 'ERROR ' + api_uri + ' ' + str(response)
    return symbol

    # quick tests
    entrez_symbol('http://www.ncbi.nlm.nih.gov/gene/672')

    entrez_symbol('http://www.ncbi.nlm.nih.gov/gene/-1')

In [11]:
# cache ncbi gene iri and symbol map
iri_sym = {}

In [12]:
# what do have for hits?
# Make a graphziz of the fa genes and their hits 
# base  edge length (weight) on motif simlarity
dot = []
dot.append("digraph FA_motif_similarity{")
dot.append('rankdir="LR"')
cluster = -1
for fas in fa_sets:
    cluster+=1
    # print(fas +' set has', '(' + str(len(fa_sets[fas])) +') genes')
    dot.append('subgraph cluster_' + str(cluster) + ' {')
    dot.append('label="' + fas +'";') 
    for fa in fa_sets[fas]:
        fa_iri = fa_sets[fas][fa]['iri']
        iri_sym[fa_iri] = entrez_symbol(fa_iri)
        
        if 'hits' in fa_sets[fas][fa] and len(fa_sets[fas][fa]['hits']) > 0:
            # print(
            #   '  ', iri_sym[fa], 
            #   ' has(' + str(len(fa_sets[fas][fa]['hits'])) +') associations')
        
            for c in fa_sets[fas][fa]['hits']:
                # try to be nice to our friends at HGNC
                if c not in iri_sym:
                    iri_sym[c] = entrez_symbol(c)
                symbol = iri_sym[c]
                dot.append('"'+ iri_sym[fa_iri]+'" -> "'+symbol+'" [weight="'+str(int(10*float(fa_sets[fas][fa]['hits'][c]['score'])+1))+'"];')
                fa_sets[fas][fa]['hits'][c]['symbol'] = symbol
        # else: 
           #print( iri_sym[fa], '\tNo hits found')
    dot.append("}")
    
dot.append("}")    

In [13]:
# output a graphviz dot file of the data structure
dotout = open('FA_motif_similarity.gv', 'w')
dotout.write( "\n".join(dot))
dotout.close()

In [15]:
# and your giant bitmap too
os.system('dot -Tpng FA_motif_similarity.gv > FA_motif_similarity.png');

In [14]:
# and dump as json
jsonout = open('fa_sets.json', 'w')
json.dump(fa_sets, jsonout)
jsonout.close()

In [15]:
# and write out a flat tab file 
tab = []
tab.append('\t'.join(['#fa_symbol','hit_symbol','score','fa_set','fa_iri','hit_iri']))
for fas in fa_sets:
    for fa_curie in fa_sets[fas]:
        for hit_iri in fa_sets[fas][fa_curie]['hits']:
            tab.append('\t'.join([
                fa_sets[fas][fa_curie]['symbol'], 
                fa_sets[fas][fa_curie]['hits'][hit_iri]['symbol'],
                fa_sets[fas][fa_curie]['hits'][hit_iri]['score'],
                fas,
                fa_sets[fas][fa_curie]['iri'],
                hit_iri
            ]))

In [16]:
tabout = open('fa_sets.tab', 'w')
tabout.write('\n'.join(tab))
tabout.close()

Currently 13 of the 27 FA genes have TFBS motif similarity associations.   
from the graph we can see which occur more than once and which are similar to other FA genes  
The ones that occur more than once may be better candidates or just genes with extra boring uprtream regions.  
I would have been slightly dissapointed if there were no hits between FA genes as it is I still whish there had been more. 